In [2]:
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Embedding, Flatten, Dense

    
    data = pd.read_csv("/train.txt", sep=';')
    data.columns = ["Text", "Emotions"]
    print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [3]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

#Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [5]:
sequences = tokenizer.texts_to_sequences(texts)
# sequences
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [6]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

print("Label classes:", label_encoder.classes_) 

Label classes: ['anger' 'fear' 'joy' 'love' 'sadness' 'surprise']


In [7]:
# one-hot encode the labels
one_hot_labels = to_categorical(labels)

In [8]:
# split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                               one_hot_labels,
                                               test_size=0.2)

In [9]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                   output_dim=128))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.3824 - loss: 1.5192 - val_accuracy: 0.7119 - val_loss: 0.8952
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8741 - loss: 0.4343 - val_accuracy: 0.8253 - val_loss: 0.5300
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9827 - loss: 0.0654 - val_accuracy: 0.8184 - val_loss: 0.5718
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9949 - loss: 0.0248 - val_accuracy: 0.8234 - val_loss: 0.5874
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9960 - loss: 0.0181 - val_accuracy: 0.8153 - val_loss: 0.6859
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9973 - loss: 0.0113 - val_accuracy: 0.8194 - val_loss: 0.6548
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9979 - loss: 0.0110 - val_accuracy: 0.8178 - val_loss: 0.6988
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9970 - loss: 0.0113 - val_accu

In [10]:
input_text = "I just got promoted at work, I'm so happy!"
# input_text = "I can't stop crying since she left me alone."
# input_text = "That noise in the dark really scared me."


# Preprocessing the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_index = np.argmax(prediction[0])
predicted_label = label_encoder.inverse_transform([predicted_index])
print(predicted_label[0])  


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
joy
